[View in Colaboratory](https://colab.research.google.com/github/Kaliostrogoblin/GRID-2018-Workshop/blob/master/Deep_learning_workshop.ipynb)

# Deep Learning for beginners with exciting example of computing poetry